# Review Instructions

Please review the MSv4 `antenna_xds` schema and the XRADIO interface (`ps['MSv4_name'].antenna_xds`). Note that the PS (processing set) interface or the main_xds should not be reviewed.

The `antenna_xds` schema specification: https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=257301047

## Preparatory Material
Go over Xarray nomenclature and selection syntax:
- https://docs.xarray.dev/en/latest/user-guide/terminology.html
- https://docs.xarray.dev/en/latest/user-guide/indexing.html

MSv2 and CASA documentation:
- MSv2 schema: https://casacore.github.io/casacore-notes/229.pdf
- MSv3 schema: https://casacore.github.io/casacore-notes/264.pdf

## `antenna_xds` Schema
The ANTENNA, FEED, and INTERFEROMETER_MODEL (VLBI) tables in the MSv2 contain closely related information:

- ANTENNA:
- FEED:
- INTERFEROMETER_MODEL (VLBI): (single field and spectral window)


Use cases:

## Key Questions to Answer
### Schema Questions
- 1.1) Are there missing use cases?
- 1.2) Is all the information present needed for offline processing?
- 1.3) Should we get rid of antenna_ids and move to just using antenna_name + "_" + station (this would also require a change to main_xds)? This would simplify doing baseline parallelism over multiple converted MS v2s since no reindexing would be required.
- 1.4) (VLBI) Instead of storing BASELINE_REFERENCE in main_xds can we store it in the antenna_xds? This would assume that for the duration of the MS v4 that the reference antennas remain constant.
- 1.5) Is the order of the dims correct (antenna_id)?
- 1.6) Should BEAM_OFFSET be sky_dir_label (Ra, Dec) or local_sky_label (Az, Alt)?
- 1.7) Do we need time dimension with BEAM_OFFSET, FEED_OFFSET, RECEPTOR_ANGLE, 
- 1.8) Should we add prefex to organize data variables? For example PHASE_DELAY -> VLBI_PHASE_DELAY?
- 1.9) Should we include the POLARIZATION_RESPONSE doesn't seem to be used?

  
### XRADIO
2.1) After reviewing the XARRAY documentation and the descriptions of the data variables in the `antenna_xds` schema, do you find the XARRAY interface intuitive and easy to use?


# Environment instructions

It is recommended to use the conda environment manager to create a clean, self-contained runtime where xradio and all its dependencies can be installed:

```bash
conda create --name xradio python=3.11 --no-default-packages
conda activate xradio
```

Clone the repository, checkout the review branch and do a local install:

```bash
git clone https://github.com/casangi/xradio.git
git checkout 168-review-ms_xdsattrsantenna_xds-schema-and-xradio-interface
cd xradio
pip install -e .
```

On macOS it is required to pre-install python-casacore using ```bash conda install -c conda-forge python-casacore```.

# Download Data

In [1]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
from xradio.vis.read_processing_set import read_processing_set
import graphviper

graphviper.utils.data.download(file="VLBA_TL016B_split_lsrk.ms")

[2024-08-08 13:16:03,388]  WARNING  graphviper:  File exists: /Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11/site-packages/graphviper/utils/data/.dropbox 
[2024-08-08 13:16:03,388]     INFO  graphviper:  Updating file metadata information ...  
 

 Download List              
 ─────────────────────────── 
  VLBA_TL016B_split_lsrk.ms

[2024-08-08 13:16:04,243]     INFO  graphviper:  File exists: VLBA_TL016B_split_lsrk.ms 


# Start Dask cluster 
Choose an approriate number of cores and memory_limit (this is per core).

In [2]:
from graphviper.dask.client import local_client

viper_client = local_client(cores=4, memory_limit="4GB")
viper_client

[2024-08-08 13:16:04,337]     INFO  graphviper:  Checking parameter values for client.local_client 
[2024-08-08 13:16:04,337]     INFO  graphviper:  Module path: /Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11//site-packages/ 
[2024-08-08 13:16:04,340]  WARNING      client:  It is recommended that the local cache directory be set using the dask_local_dir parameter. 
[2024-08-08 13:16:05,047]     INFO      client:  Created client <MenrvaClient: 'tcp://127.0.0.1:63088' processes=4 threads=4, memory=14.90 GiB> 


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63088,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:63099,Total threads: 1
Dashboard: http://127.0.0.1:63103/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:63091,


# Convert dataset

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

in_file = "VLBA_TL016B_split_lsrk.ms"
out_file = "VLBA_TL016B_split_lsrk.vis.zarr"

convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    parallel=False,
    overwrite=True,
)

[2024-08-08 13:16:05,073]     INFO      client:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-08-08 13:16:05,080]     INFO      client:  Number of partitions: 4 
[2024-08-08 13:16:05,080]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [-1], FIELD [0], SCAN [0] 
[2024-08-08 13:16:05,250]  WARNING      client:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-08 13:16:05,344]     INFO      client:  OBSERVATION_ID [0], DDI [0], STATE [-1], FIELD [1], SCAN [0] 
[2024-08-08 13:16:05,561]  WARNING      client:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-08 13:16:05,647]     INFO      client:  OBSERVATION_ID [0], DDI [1], STATE [-1], FIELD [0], SCAN [0] 
[2024-08-08 13:16:05,836]  WARNING      client:  Source_id is -1. No source information will be included in the field_and_source_xds. 
[2024-08-08 13:16:05,945]     INFO      client:  OBSERV

In [4]:
import numpy as np
a = np.array([1, 2, 3, 4, 5])
b = np.array([5, 4, 3, 2, 1])

np.concatenate([a, b])


array([1, 2, 3, 4, 5, 5, 4, 3, 2, 1])

# Inspect Processing Set

In [5]:
import pandas as pd

# Set the maximum number of rows displayed before scrolling
pd.set_option("display.max_rows", 1000)

from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set("VLBA_TL016B_split_lsrk.vis.zarr")
ps.summary()

,name,obs_mode,shape,polarization,spw_name,field_name,source_name,field_coords,start_frequency,end_frequency
0,VLBA_TL016B_split_lsrk_3,obs_0,"(540, 55, 6, 2)","[RR, LL]",spw_1,[J1154+6022_1],[Unknown],"[fk5, 11h54m04.54s, 60d22m20.82s]",5.068199e+09,5.070699e+09
1,VLBA_TL016B_split_lsrk_2,obs_0,"(200, 55, 6, 2)","[RR, LL]",spw_1,[4C39.25_0],[Unknown],"[fk5, 9h27m03.01s, 39d02m20.85s]",5.068199e+09,5.070699e+09
2,VLBA_TL016B_split_lsrk_0,obs_0,"(200, 55, 6, 2)","[RR, LL]",spw_0,[4C39.25_0],[Unknown],"[fk5, 9h27m03.01s, 39d02m20.85s]",5.004196e+09,5.006697e+09
3,VLBA_TL016B_split_lsrk_1,obs_0,"(540, 55, 6, 2)","[RR, LL]",spw_0,[J1154+6022_1],[Unknown],"[fk5, 11h54m04.54s, 60d22m20.82s]",5.004196e+09,5.006697e+09


# Inspect antenna_xds:

In [6]:
ant_xds = ps['VLBA_TL016B_split_lsrk_0'].attrs['antenna_xds'].load()
ant_xds

<xarray.Dataset> Size: 521kB
Dimensions:                   (name: 10, cartesian_pos_label: 3,
                               receptor_name: 2, sky_dir_label: 2,
                               antenna_id: 10, gain_curve_time: 1,
                               poly_term: 1, phase_cal_time: 642, tone_label: 2)
Coordinates:
  * cartesian_pos_label       (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
  * gain_curve_time           (gain_curve_time) float64 8B 5.152e+09
    mount                     (name) <U6 240B 'ALT-AZ' 'ALT-AZ' ... 'ALT-AZ'
  * name                      (name) <U2 80B 'BR' 'FD' 'HN' ... 'OV' 'PT' 'SC'
  * phase_cal_time            (phase_cal_time) float64 5kB 5.152e+09 ... 5.15...
  * receptor_name             (receptor_name) <U21 168B '0' '1'
  * sky_dir_label             (sky_dir_label) <U3 24B 'ra' 'dec'
    station                   (name) <U2 80B 'BR' 'FD' 'HN' ... 'OV' 'PT' 'SC'
Dimensions without coordinates: antenna_id, poly_term, tone_label
Data variables:
    ANTENNA_DISH_DIAMETER     (name) float64 80B 25.0 25.0 25.0 ... 25.0 25.0
    ANTENNA_FEED_OFFSET       (name, cartesian_pos_label) float64 240B 2.13 ....
    ANTENNA_POSITION          (name, cartesian_pos_label) float64 240B -2.112...
    BEAM_OFFSET               (name, receptor_name, sky_dir_label) float64 320B ...
    GAIN_CURVE                (antenna_id, gain_curve_time, poly_term, receptor_name) float32 80B ...
    GAIN_CURVE_INTERVAL       (antenna_id, gain_curve_time) float64 80B 2.601...
    PHASE_CAL                 (antenna_id, phase_cal_time, receptor_name, tone_label) complex64 205kB ...
    PHASE_CAL_CABLE_CAL       (antenna_id, phase_cal_time) float64 51kB 1.507...
    PHASE_CAL_INTERVAL        (antenna_id, phase_cal_time) float64 51kB 30.0 ...
    PHASE_CAL_TONE_FREQUENCY  (antenna_id, phase_cal_time, receptor_name, tone_label) float64 205kB ...
    POLARIZATION_TYPE         (name, receptor_name) <U1 80B 'R' 'L' ... 'R' 'L'
    RECEPTOR_ANGLE            (name, receptor_name) float64 160B 0.0 0.0 ... 0.0
Attributes:
    overall_telescope_name:  VLBA
    relocatable_antennas:    False

In [7]:
ant_xds.mount

<xarray.DataArray 'mount' (name: 10)> Size: 240B
array(['ALT-AZ', 'ALT-AZ', 'ALT-AZ', 'ALT-AZ', 'ALT-AZ', 'ALT-AZ',
       'ALT-AZ', 'ALT-AZ', 'ALT-AZ', 'ALT-AZ'], dtype='<U6')
Coordinates:
    mount    (name) <U6 240B 'ALT-AZ' 'ALT-AZ' 'ALT-AZ' ... 'ALT-AZ' 'ALT-AZ'
  * name     (name) <U2 80B 'BR' 'FD' 'HN' 'KP' 'LA' 'MK' 'NL' 'OV' 'PT' 'SC'
    station  (name) <U2 80B 'BR' 'FD' 'HN' 'KP' 'LA' 'MK' 'NL' 'OV' 'PT' 'SC'